# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import libraries

In [ ]:
!pip install krippendorff

In [2]:
import krippendorff
import numpy as np
import pandas as pd
# Prevent AttributeError. via
# https://stackoverflow.com/a/76404841
pd.DataFrame.iteritems = pd.DataFrame.items
from sklearn.metrics import f1_score

In [3]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("dplyr")
install.packages("ggh4x")
devtools::install_github("hadley/ggplot2")
devtools::install_github("cran/GiniWegNeg")
devtools::install_github("kgwet/irrCAC")
install.packages("patchwork")
install.packages("showtext")
install.packages("stringr")
library(dplyr)
library(ggh4x)
library(ggplot2)
library(GiniWegNeg)
library(irrCAC)
library(patchwork)
library(showtext)
# Need to download Arial.ttf and upload it to your Google Drive prior.
font_add("Arial", regular = "/content/drive/MyDrive/Arial.ttf")
library(stringr)

# Import datasets

In [5]:
COVID_HATE_0301 = pd.read_csv(
    '/content/drive/MyDrive/datasets/cleaning/'
    'COVID-HATE_predictions_cleaned_gpt-3.5-turbo-0301.tsv',
    sep='\t')
COVID_HATE_0301 = COVID_HATE_0301.reset_index(drop=True)
COVID_HATE_0301 = COVID_HATE_0301.astype('string')
COVID_HATE_0613 = pd.read_csv(
    '/content/drive/MyDrive/datasets/cleaning/'
    'COVID-HATE_predictions_cleaned_gpt-3.5-turbo-0613.tsv',
    sep='\t')
COVID_HATE_0613 = COVID_HATE_0613.reset_index(drop=True)
COVID_HATE_0613 = COVID_HATE_0613.astype('string')

# Compute macro F1-scores

In [ ]:
def compute_F1(df):
    for i in df.iloc[:, 3:].columns:
        print(i)
        print(np.round(f1_score(df['label'],
                                df[i],
                                average='macro'),
                       3))


compute_F1(COVID_HATE_0301)
print()
compute_F1(COVID_HATE_0613)

# Define a function to compute and interpret Krippendorff's alpha coefficients

In [7]:
%%R
compute_alpha <- function(df, reference, model, measure) {
    alpha_list <- c()
    for (i in seq_along(colnames(df))) {
        print(colnames(df)[i])
        # Retrieve and interpret Krippendorff's alpha values. via
        # https://cran.r-project.org/web/packages/irrCAC/vignettes/
        # benchmarking.html
        a <- krippen.alpha.raw(df[, c(reference, colnames(df)[i])])
        print(a$est)
        print(a$categories)
        print(landis.koch.bf(a$est$coeff.val, a$est$coeff.se))
        if (i > 1) {
            alpha_list[(i - 1)] <- a$est$coeff.val
        }
    }
    res <- data.frame(
        alpha = alpha_list,
        persona = colnames(df[, -1]),
        model = rep(model, length(alpha)),
        measure = rep(measure, length(alpha))
    )
    return(res)
}

# Call the `compute_alpha` function to compute and interpret Krippendorff's alpha coefficients between the personas and the no-persona defaults

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i COVID_HATE_0301
alpha_ib_0301 <- compute_alpha(COVID_HATE_0301[, 4:13], "predicted_labels_base", "gpt-3.5-turbo-\"0301\"", "\"alpha\"[ib]")
print(alpha_ib_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i COVID_HATE_0613
alpha_ib_0613 <- compute_alpha(COVID_HATE_0613[, 4:13], "predicted_labels_base", "gpt-3.5-turbo-\"0613\"", "\"alpha\"[ib]")
print(alpha_ib_0613)

# Call the `compute_alpha` function to compute and interpret Krippendorff's alpha coefficients between personas and actual annotators

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i COVID_HATE_0301
alpha_ia_0301 <- compute_alpha(COVID_HATE_0301[, c(3, 12:13)], "label", "gpt-3.5-turbo-\"0301\"", "\"alpha\"[ia]")
print(alpha_ia_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i COVID_HATE_0613
alpha_ia_0613 <- compute_alpha(COVID_HATE_0613[, c(3, 12:13)], "label", "gpt-3.5-turbo-\"0613\"", "\"alpha\"[ia]")
print(alpha_ia_0613)

# Plot Krippendorff's alpha coefficients and Bias

In [16]:
%%R
showtext_auto()
data <- rbind(alpha_ib_0301, alpha_ib_0613, alpha_ia_0301, alpha_ia_0613)
# Replace spaces with underscores. via
# https://stackoverflow.com/a/53107084
data$persona <- gsub("_", " ", data$persona)
# Remove first few words. via
# https://stackoverflow.com/a/64889117
data$persona <- sub("^\\w+\\s\\w+\\s\\w+\\s", "", data$persona)
# Remove everything after specified substring. via
# https://stackoverflow.com/a/53503384
data$persona <- sub("in the.+$", "", data$persona)
# Truncate character strings after certain number of characters. via
# https://stackoverflow.com/a/46759426
data$persona <- ifelse(nchar(data$persona) > 18,
    paste0(substring(data$persona, 1, 16), " ..."),
    data$persona
)
# Compute bias
data$bias <- c(
    rep(Gini_RSV(alpha_ib_0301$alpha[1:7])[[1]], 7),
    rep(Gini_RSV(alpha_ib_0301$alpha[8:9])[[1]], 2),
    rep(Gini_RSV(alpha_ib_0613$alpha[1:7])[[1]], 7),
    rep(Gini_RSV(alpha_ib_0613$alpha[8:9])[[1]], 2),
    rep(0, 4)
)
data$labs <- c(
    rep(format(round(Gini_RSV(alpha_ib_0301$alpha[1:7])[[1]], digits = 3),
        nsmall = 3
    ), 7),
    rep(format(round(Gini_RSV(alpha_ib_0301$alpha[8:9])[[1]], digits = 3),
        nsmall = 3
    ), 2),
    rep(format(round(Gini_RSV(alpha_ib_0613$alpha[1:7])[[1]], digits = 3),
        nsmall = 3
    ), 7),
    rep(format(round(Gini_RSV(alpha_ib_0613$alpha[8:9])[[1]], digits = 3),
        nsmall = 3
    ), 2),
    rep("NA", 4)
)
data$attribute <- c(
    rep("Race/Nationality", 7),
    rep("Annotator~demographic", 2),
    rep("Race/Nationality", 7),
    rep("Annotator~demographic", 6)
)
data$grp <- c(rep(1, 22))
# Dot plots. via
# https://uc-r.github.io/cleveland-dot-plots
g1 <- ggplot(
    data,
    aes(alpha, grp)
) +
    geom_line(aes(group = grp),
        size = 0.05
    ) +
    geom_point(size = 0.1) +
    # Label max value dots. via
    # https://stackoverflow.com/a/48351513
    geom_text(
        data = . %>% group_by(measure, attribute, model) %>%
            filter(alpha == max(alpha)),
        # Wrap and truncate text. via
        # https://stackoverflow.com/a/73357493
        aes(label = str_wrap(persona,
            width = 8
        )),
        hjust = 0,
        family = "Arial",
        size = 7 / .pt,
        nudge_x = 0.01,
        lineheight = 1
    ) +
    xlim(0, 1) +
    xlab("Krippendorff's alpha coefficient") +
    facet_nested(
        rows = vars(measure, attribute, model),
        switch = "y",
        labeller = label_parsed,
        strip = strip_nested(size = "variable")
    ) +
    theme(
        axis.text.y = element_blank(),
        axis.title.y = element_blank(),
        axis.ticks.y = element_blank(),
        axis.ticks = element_line(size = 0.1),
        axis.text = element_text(
            family = "Arial",
            size = 7
        ),
        strip.text.y.left = element_text(
            angle = 0,
            family = "Arial",
            size = 7
        ),
        strip.background = element_rect(
            colour = "white",
            fill = "#ECECEC"
        ),
        panel.grid = element_line(
            color = "black",
            size = 0.1,
            linetype = 2
        ),
        panel.grid.major.y = element_blank(),
        panel.grid.minor.y = element_blank(),
        panel.grid.minor.x = element_blank(),
        panel.spacing.y = unit(0, "lines"),
        panel.background = element_rect(
            color = "black",
            fill = "white",
            linewidth = 0.1
        ),
        text = element_text(
            family = "Arial",
            size = 7
        )
    )
data$alpha <- NULL
data$persona <- NULL
data <- data[!duplicated(data), ]
g2 <- ggplot(
    data,
    aes(grp, bias)
) +
    geom_text(aes(label = labs),
        hjust = -0.2,
        family = "Arial",
        size = 7 / .pt
    ) +
    coord_flip() +
    ylab("Bias") +
    scale_y_continuous(
        breaks = c(0.00, 1.00),
        limits = c(0, 1)
    ) +
    geom_bar(stat = "identity", fill = "black") +
    facet_nested(
        rows = vars(measure, attribute, model),
        labeller = label_parsed
    ) +
    theme(
        axis.text.y = element_blank(),
        axis.title.y = element_blank(),
        axis.ticks.y = element_blank(),
        axis.ticks = element_line(size = 0.1),
        axis.text = element_text(
            family = "Arial",
            size = 7
        ),
        strip.text = element_blank(),
        strip.background = element_blank(),
        panel.grid = element_line(
            color = "black",
            size = 0.1,
            linetype = 2
        ),
        panel.grid.major.y = element_blank(),
        panel.grid.minor.y = element_blank(),
        panel.grid.minor.x = element_blank(),
        panel.spacing.y = unit(0, "lines"),
        panel.background = element_rect(
            color = "black",
            fill = "white",
            linewidth = 0.1
        ),
        text = element_text(
            family = "Arial",
            size = 7
        )
    )
g1 + g2 + plot_layout(widths = c(9, 1))
ggsave(
    "/content/drive/MyDrive/datasets/cleaning/COVID-HATE.eps",
    height = 6,
    width = 18,
    units = "cm"
)

# Double check Krippendorff's alpha coefficients

In [ ]:
def double_check_alpha(df, reference):
    for i in df.columns:
        print(i)
        print(np.round(krippendorff.alpha(
            df[[reference, i]].T.to_numpy().tolist(),
            level_of_measurement='nominal'), 3))


double_check_alpha(COVID_HATE_0301.iloc[:, 3:], 'predicted_labels_base')
print()
double_check_alpha(COVID_HATE_0613.iloc[:, 3:], 'predicted_labels_base')
print()
double_check_alpha(COVID_HATE_0301.iloc[:, [2, 11, 12]], 'label')
print()
double_check_alpha(COVID_HATE_0613.iloc[:, [2, 11, 12]], 'label')